In [12]:
import fitz  # PyMuPDF
import pandas as pd
import numpy as np
import os

# PDFファイルのパス
# 使用例
# 対象のPDFファイルを指定
#pdf_file = "ir/2025/34070_旭化成/24jp.pdf"  
base_dir = "ir/2025/"
#base_dir = "ir/2024/"

#pdf_dir =  "34070_旭化成"
#pdf_filename = "24jp.pdf"
pdf_dir =  "40040_レゾナック・ホールディングス"
pdf_filename = "pdf-sustainability-report-integratedreport-RESONAC24J_spread.pdf"
#pdf_dir =  "41880_三菱ケミカルグループ"
#pdf_filename = "23.pdf"
pdf_path = base_dir + pdf_dir + "/" + pdf_filename
info_path  = base_dir + pdf_dir + "/info"

output_header_path = info_path + "/" + "header_patterns.csv"
output_footer_path = info_path + "/" + "footer_patterns.csv"
margin_path = info_path + "/" + "margin.csv"

def extract_blocks_from_pdf(pdf_path):
    pdf_doc = fitz.open(pdf_path)
    data = []

    # 各ページからデータ抽出
    for page_num in range(len(pdf_doc)):
        page = pdf_doc[page_num]
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            block_bbox = block["bbox"]
            for line in block.get("lines", []):
                line_text = " ".join([span["text"] for span in line["spans"]])
                line_bbox = line["bbox"]
                data.append({
                    "page": page_num + 1,
                    "x0": line_bbox[0],
                    "y0": line_bbox[1],
                    "x1": line_bbox[2],
                    "y1": line_bbox[3],
                    "text": line_text.strip()
                })
    pdf_doc.close()
    return pd.DataFrame(data)

def identify_header_footer(data, threshold=40):
    # ページごとの y0 値の分布を取得
    headers = []
    footers = []

    for page_num, group in data.groupby("page"):
        y0_values = group["y0"].values
        if len(y0_values) == 0:
            continue

        min_y0, max_y0 = np.min(y0_values), np.max(y0_values)

        # ヘッダ候補: y0 が分布の下位 (小さい値)
        header_candidates = group[np.abs(group["y0"] - min_y0) < threshold]
        headers.append(header_candidates)

        # フッタ候補: y0 が分布の上位 (大きい値)
        footer_candidates = group[np.abs(group["y0"] - max_y0) < threshold]
        footers.append(footer_candidates)

    # ヘッダ・フッタを統合
    headers = pd.concat(headers, ignore_index=True)
    footers = pd.concat(footers, ignore_index=True)

    # 頻出パターンを確認 (x0, y0, text を基準にグループ化)
    header_patterns = headers.groupby(["x0", "y0", "text"]).size().reset_index(name="count")
    footer_patterns = footers.groupby(["x0", "y0", "text"]).size().reset_index(name="count")

    return header_patterns, footer_patterns

def main():
    # PDFデータを抽出
    pdf_data = extract_blocks_from_pdf(pdf_path)

    # ヘッダ・フッタを識別
    headers, footers = identify_header_footer(pdf_data)

    # 結果をソートして表示
    headers_sorted = headers.sort_values("count", ascending=False)
    footers_sorted = footers.sort_values("count", ascending=False)

    print("Identified Header Patterns:")
    print(headers_sorted)

    print("\nIdentified Footer Patterns:")
    print(footers_sorted)

    # ソート済み結果をCSVに保存
    headers_sorted.to_csv(output_header_path, index=False, encoding="utf-8-sig")
    footers_sorted.to_csv(output_footer_path, index=False, encoding="utf-8-sig")

    #margin.csvが存在しない場合には、ブランクのcsvを書き込み
# チェックと生成
    if not os.path.exists(margin_path):
        # データを作成 (全てNone)
        data = {
            "header_y": [None],
            "footer_y": [None],
            "left_margin_x": [None],
            "right_margin_x": [None]
        }
        df_margin = pd.DataFrame(data)
    
        # CSVファイルを保存
        df_margin.to_csv(margin_path, index=False)
        print(f"{margin_path} を新たに作成しました。")
    else:
        print(f"{margin_path} は既に存在しています。")    
    

if __name__ == "__main__":
    main()


Identified Header Patterns:
             x0         y0        text  count
69  1062.807251  14.761236           ン     64
46   733.712402  14.761236      クの自己紹介     64
57   934.919312  14.761236           ・     64
59   940.245605  14.761236     ガバナンス改革     64
53   897.561646  14.761236       コーポレー     64
..          ...        ...         ...    ...
33   420.707977  49.841599      活動の詳細は      1
34   525.054016  73.700592  ─半導体材料事業への      1
1     42.347801  40.158421         Now      1
36   543.281311  63.066734    最大化への道すじ      1
35   541.435974  52.441002   レゾナックの稼ぐ力      1

[70 rows x 4 columns]

Identified Footer Patterns:
              x0          y0                     text  count
617   692.081787  612.728149                     64 ％      2
592   663.281982  613.915649                     51 ％      2
662   728.518799  613.915649                     51 ％      2
960  1067.391602  651.858154                       18      2
717   785.854126  620.825134                      （年）      2
..